In [ ]:
import numpy as np
import rasterio
from rasterio.warp import(
    calculate_default_transform, 
    reproject, 
    Resampling
)
from rasterio import Affine
import rioxarray
import xarray as xr

In [ ]:
# function for tif reprojection and write
def reproject_tif(filepath, dst_filepath, dst_crs, resampling=Resampling.bilinear):

    with rasterio.open(filepath) as src:
        transform, width, height = calculate_default_transform(
            src.crs,
            dst_crs,
            src.width,
            src.height,
            *src.bounds
        )

        kwargs = src.meta.copy()
        kwargs.update({ 
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        with rasterio.open(dst_filepath, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    rasterio.band(src, i), 
                    rasterio.band(dst, i), 
                    src_transform=src.transform, 
                    src_crs=src.crs, 
                    dst_transform=transform, 
                    dst_crs=dst_crs, 
                    resampling=resampling
                )
reproject_tif('../Downloads/elevation.tif', './elevation_2163.tif', {'init': 'EPSG:2163'})

In [ ]:
# function for DataArray reprojection and return
elev_4326 = xr.open_rasterio('../Downloads/elevation.tif')
src_height, src_width = elev_4326.shape[-2], elev_4326.shape[-1]

left, right = float(elev_4326['x'][0].data), float(elev_4326['x'][-1].data)
bottom, top = float(elev_4326['y'][-1].data), float(elev_4326['y'][0].data)

dst_crs = {'init': 'EPSG:3857'}

with rasterio.Env():
    transform, height, width = calculate_default_transform(
        elev_4326.crs,
        dst_crs,
        src_width,
        src_height,
        left=left,
        bottom=bottom,
        right=right,
        top=top
    )
    
    destination = np.zeros((width, height), np.uint8)
    
    reproject(
        elev_4326.data,
        destination,
        src_transform=elev_4326.transform,
        src_crs=elev_4326.crs,
        dst_transform=transform,
        dst_crs=dst_crs,
        resampling=Resampling.nearest
    )
    
    kwargs = {
        'dtype': rasterio.uint8,
        'count': 1,
        'crs': dst_crs,
        'transform': transform,
        'height': height,
        'width': width,
    }
    
    destination = xr.DataArray(destination, dims=('y', 'x'))

destination = destination.expand_dims('band')
destination['band'] = elev_4326['band']

with rasterio.open('./elevation_da_tif_3857.tif', 'w', **kwargs) as dst:
    dst.write(destination[0].data, 1)
    

In [ ]:
# higher level with rioxarray

def reproject_xr_da(da, dst_crs, resolution=None,
                    shape=None, transform=None,
                    resampling=<Resampling.nearest: 0>):
    return da.rio.reproject(dst_crs, resolution=resolution,
                            shape=shape, transform=transform,
                            resampling=resampling)